In [ ]:
import json
from dateutil.parser import parse
import pprint

In [ ]:
f = open('fhir/Candis134_Wolff180_caac926a-429a-422d-9333-c9a3a04e68f9.json', 'r')
text = f.read()
f.close()
print(type(text))

In [ ]:
with open('fhir/Candis134_Wolff180_caac926a-429a-422d-9333-c9a3a04e68f9.json') as f:
    bundle = json.load(f)
print(type(bundle))

In [ ]:
for entry in bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    if resource_type == 'Claim':
        id = resource['id']
        patient = resource['patient']['reference']
        status = resource['status']
        total = resource['total']['value']
        currency = resource['total']['currency']
        start = resource['billablePeriod']['start']
        end = resource['billablePeriod']['end']
        print(patient, status, total, currency, start, end)
        break

In [ ]:
for entry in bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    if resource_type == 'Claim':
        id = resource['id']
        patient = resource['patient']['reference']
        status = resource['status']
        total = resource['total']['value']
        currency = resource['total']['currency']
        start = parse(resource['billablePeriod']['start'])
        end = parse(resource['billablePeriod']['end'])
        print(patient, status, total, currency, start, end)
        break

In [ ]:
import sqlalchemy as db

In [ ]:
engine = db.create_engine('sqlite:///fhir.sqlite')
connection = engine.connect()
metadata = db.MetaData()

In [ ]:
claims = db.Table('claims', metadata,
                  db.Column('id', db.Integer()),
                  db.Column('patient', db.String(255)),
                  db.Column('status', db.String(255)),
                  db.Column('total', db.Float()),
                  db.Column('currency', db.String(255)),
                  db.Column('start', db.DateTime()),
                  db.Column('end', db.DateTime()))

metadata.create_all(engine) #Creates the table

In [ ]:
print(claims.columns.keys())

In [ ]:
for entry in bundle['entry']:
    resource = entry['resource']
    resource_type = resource['resourceType']
    if resource_type == 'Claim':
        id = resource['id']
        patient = resource['patient']['reference']
        status = resource['status']
        total = resource['total']['value']
        currency = resource['total']['currency']
        start = parse(resource['billablePeriod']['start'])
        end = parse(resource['billablePeriod']['end'])        
        query = db.insert(claims).values(id=id, 
                                         patient=patient,
                                         status=status,
                                         total=total,
                                         currency=currency,
                                         start=start,
                                         end=end)

        result_proxy = connection.execute(query)

In [ ]:
stmt = db.select([claims])
results = connection.execute(stmt).fetchall()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.columns = results[0].keys()

In [ ]:
df.head()

In [ ]:
stmt = db.select([claims])
results = connection.execute(stmt).fetchall()
pd.DataFrame(results).head()
df.columns = results[0].keys()
df.head()

In [ ]:
stmt = db.select([claims]).where(claims.columns.total >= 25000.0).order_by(db.desc(claims.columns.total))
results = connection.execute(stmt).fetchall()
df = pd.DataFrame(results).head()
df.columns = results[0].keys()
df.head()

In [ ]:
stmt = db.select([db.func.sum(claims.columns.total).label('total claims'),
                  db.func.avg(claims.columns.total).label('average claim')])
results = connection.execute(stmt).fetchall()
df = pd.DataFrame(results).head()
df.columns = results[0].keys()
df.head()